In [1]:
import pandas as pd 
import random
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)
import nltk 
import gensim
import logging

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

# for LSTM (keras with tf backend)
import gzip
import os
import pickle
import requests
import time
import re
os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.initializers import he_normal
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using CNTK backend


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/edithzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/anaconda/envs/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:25: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [5]:
vizwiz_features_train_color = pd.read_csv('azure_features_images/data/vizwiz_train_color_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                    quotechar='"', error_bad_lines=False)
vizwiz_features_train_text = pd.read_csv('azure_features_images/data/vizwiz_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_features_val_color = pd.read_csv('azure_features_images/data/vizwiz_val_color_recognition.csv',
                                  delimiter=';', engine='python',
                                  dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                  quotechar='"', error_bad_lines=False)
vizwiz_features_val_text = pd.read_csv('azure_features_images/data/vizwiz_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vqa_features_train_color = pd.read_csv('azure_features_images/data/vqa_train_color_recognition.csv',
                                 delimiter=';', engine='python', 
                                 dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                 quotechar='"', error_bad_lines=False)
vqa_features_train_text = pd.read_csv('azure_features_images/data/vqa_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)
vqa_features_val_color = pd.read_csv('azure_features_images/data/vqa_val_color_recognition.csv',
                               delimiter=';', engine='python',
                               dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                               quotechar='"', error_bad_lines=False)
vqa_features_val_text = pd.read_csv('azure_features_images/data/vqa_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_targets_train = pd.read_csv('../vizwiz_skill_typ_train.csv', dtype={'QID':str},
                                   delimiter=',', quotechar='"',
                                   engine='python', error_bad_lines=False)
vizwiz_targets_val = pd.read_csv('../vizwiz_skill_typ_val.csv', dtype={'QID':str},
                                 delimiter=',', quotechar='"', engine='python', error_bad_lines=False)
vqa_targets_train = pd.read_csv('../vqa_skill_typ_train.csv', dtype={'QID':str},
                               engine='python', quotechar='"', error_bad_lines=False)
vqa_targets_val = pd.read_csv('../vqa_skill_typ_val.csv', dtype={'QID':str},
                               engine='python', quotechar='"', error_bad_lines=False)

Skipping line 2150: ';' expected after '"'
Skipping line 42: Expected 4 fields in line 42, saw 7
Skipping line 60: Expected 4 fields in line 60, saw 8
Skipping line 157: Expected 4 fields in line 157, saw 5
Skipping line 207: Expected 4 fields in line 207, saw 6
Skipping line 210: Expected 4 fields in line 210, saw 5
Skipping line 275: Expected 4 fields in line 275, saw 5
Skipping line 293: Expected 4 fields in line 293, saw 5
Skipping line 341: Expected 4 fields in line 341, saw 7
Skipping line 357: Expected 4 fields in line 357, saw 5
Skipping line 376: Expected 4 fields in line 376, saw 5
Skipping line 469: Expected 4 fields in line 469, saw 6
Skipping line 478: Expected 4 fields in line 478, saw 5
Skipping line 494: Expected 4 fields in line 494, saw 5
Skipping line 496: Expected 4 fields in line 496, saw 6
Skipping line 504: Expected 4 fields in line 504, saw 5
Skipping line 512: Expected 4 fields in line 512, saw 5
Skipping line 523: Expected 4 fields in line 523, saw 6
Skipping 

In [6]:
def join_feature_target(feature_df_text, feature_df_color, target_df):
    feature_text = copy.deepcopy(feature_df_text)
    feature_color = copy.deepcopy(feature_df_color)
    target = copy.deepcopy(target_df)
    # text features 
    feature_text.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_text.set_index('QID', inplace=True)
    # color features
    feature_color.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_color.set_index('QID', inplace=True)
    # join features
    features = feature_text.join(feature_color[['descriptions','tags','dominant_colors']],
                                 on='QID',
                                 how='outer')
    # join features with target
    target = target[['QID', 'IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH']]
    target.set_index('QID', inplace=True)
    target = target.astype(dtype=str)
    df = target.join(features, on='QID', how='inner')
    df['descriptions'].astype(list)
    return df

def lem(s):
    arr = s.split(" ")
    lem = WordNetLemmatizer()
    op = ""
    for w in arr:
        word = lem.lemmatize(w) + ' '
        op += word
    return op

def preprocess_text(feature_columns):
    """ output an nparray with single document per data point """
    ip = copy.deepcopy(feature_columns).values
    op = []
    for i in range(ip.shape[0]):
        doc      =  ""
        for j in range(ip.shape[1]):
            # clean up chars
            s    =  str(ip[i][j])
            s    =  s.translate({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+'"}).lower() + " "
            if j == 1:             # clean descriptions
                s = re.sub(r'confidence\s+\d+', '', s)
                s = re.sub(r'text', '', s)
            # lexicon normalize
            s    = lem(s)
            doc  += (s.strip())
        op.append(doc)
    op = np.asarray(op)
    return op

In [8]:
vizwiz_train   = join_feature_target(vizwiz_features_train_text, 
                                   vizwiz_features_train_color, 
                                   vizwiz_targets_train)
vizwiz_val     = join_feature_target(vizwiz_features_val_text, 
                                 vizwiz_features_val_color, 
                                 vizwiz_targets_val)
vqa_train      = join_feature_target(vqa_features_train_text, 
                                   vqa_features_train_color, 
                                   vqa_targets_train)
vqa_val        = join_feature_target(vizwiz_features_val_text, 
                                 vqa_features_val_color, 
                                 vqa_targets_val)
print(vizwiz_train.shape, vqa_train.shape, vizwiz_train.shape[0] + vqa_train.shape[0])
print(vizwiz_val.shape, vqa_val.shape, vizwiz_val.shape[0] + vqa_val.shape[0])

# create X and Y

train = pd.concat([vizwiz_train, vqa_train], axis=0)
val   = pd.concat([vizwiz_val, vqa_val], axis=0)
print("Training: {}\nValidation: {}".format(train.shape, val.shape))

features_train = preprocess_text(train[['QSN','descriptions', 'tags', 'dominant_colors', 
                                        'handwritten_text', 'ocr_text']])
txt_train      = train['TXT'].values
col_train      = train['COL'].values
cnt_train      = train['CNT'].values

features_val   = preprocess_text(val[['QSN', 'descriptions', 'tags', 'dominant_colors',
                                      'handwritten_text', 'ocr_text']])
txt_val        = val['TXT'].values.astype('float32')
col_val        = val['COL'].values.astype('float32')
cnt_val        = val['CNT'].values.astype('float32')

(14257, 13) (3230, 13) 17487
(2247, 13) (513, 13) 2760
Training: (17487, 13)
Validation: (2760, 13)


In [9]:
# tokenize
tok        = Tokenizer(num_words=VOCAB_SIZE, 
                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                       lower=True,
                       split=" ")
tok.fit_on_texts(features_train)

# create sequences & pad
train_seq  = tok.texts_to_sequences(features_train)
train_seq  = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq    = tok.texts_to_sequences(features_val)
val_seq    = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

# check class distribution
text_recognition_labels = to_categorical(np.asarray(txt_train)).astype('float32')
color_recognition_labels = to_categorical(np.asarray(col_train)).astype('float32')
print('Number of samples each class - Vizwiz - train')
print('Text recognition', text_recognition_labels.sum(axis=0))
print('Color recognition', color_recognition_labels.sum(axis=0))
n_classes = 2

Number of samples each class - Vizwiz - train
Text recognition [9119. 8368.]
Color recognition [10926.  6561.]


In [10]:
# punkt sentence level tokenizer
sent_lst = []
for doc in features_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)
        
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst,
                                        min_count=6,
                                        size=EMBEDDING_DIM,
                                        sg=1,
                                        workers=os.cpu_count())

2019-02-28 03:29:27,308 : INFO : collecting all words and their counts
2019-02-28 03:29:27,309 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-28 03:29:27,371 : INFO : PROGRESS: at sentence #10000, processed 347442 words, keeping 31941 word types
2019-02-28 03:29:27,420 : INFO : collected 44967 word types from a corpus of 609446 raw words and 17487 sentences
2019-02-28 03:29:27,425 : INFO : Loading a fresh vocabulary
2019-02-28 03:29:27,460 : INFO : effective_min_count=6 retains 4665 unique words (10% of original 44967, drops 40302)
2019-02-28 03:29:27,461 : INFO : effective_min_count=6 leaves 553935 word corpus (90% of original 609446, drops 55511)
2019-02-28 03:29:27,473 : INFO : deleting the raw counts dictionary of 44967 items
2019-02-28 03:29:27,476 : INFO : sample=0.001 downsamples 65 most-common words
2019-02-28 03:29:27,477 : INFO : downsampling leaves estimated 389589 word corpus (70.3% of prior 553935)
2019-02-28 03:29:27,488 : INFO : estima

In [11]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 4665 word vectors


In [12]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 4665 word vectors


In [13]:
def lstm_create_train(labels, learning_rate, lstm_dim, batch_size, num_epochs, optimizer_param, regularization=1e-7):
    
    l2_reg = regularizers.l2(regularization)
    
    # init model
    embedding_layer = Embedding(VOCAB_SIZE,
                                EMBEDDING_DIM,
                                input_length=MAX_DOC_LEN,
                                trainable=True,
                                mask_zero=False,
                                embeddings_regularizer=l2_reg,
                                weights=[embedding_matrix])
    lstm_layer = LSTM(units=lstm_dim, kernel_regularizer=l2_reg)
    dense_layer = Dense(n_classes,
                        activation='softmax', 
                        kernel_regularizer=l2_reg)

    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(lstm_layer))
    model.add(Dropout(0.5))
    model.add(dense_layer)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer_param,
                  metrics=['acc'])
    history = History()
    csv_logger = CSVLogger('./LSTM/text/{}_{}_{}_{}.log'.format(learning_rate, regularization, batch_size, num_epochs),
                           separator=',',
                           append=True)
    t1 = time.time()
    # model fit
    model.fit(train_seq,
              labels.astype('float32'),
              batch_size=batch_size,
              epochs=num_epochs,
              callbacks=[history, csv_logger],
              verbose=2)
    t2 = time.time()
    # save hdf5
    model.save('./LSTM/text/{}_{}_{}_{}_model.h5'.format(learning_rate, regularization, batch_size, num_epochs))
    np.savetxt('./LSTM/text/{}_{}_{}_{}_time.txt'.format(learning_rate, regularization, batch_size, num_epochs), 
               [regularization, (t2-t1) / 3600])
    with open('./LSTM/text/{}_{}_{}_{}_history.txt'.format(learning_rate, regularization, batch_size, num_epochs), "w") as res_file:
        res_file.write(str(history.history))

In [14]:
L = 1e-1
R = 0
B = 32
E = 30
lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=100, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))

Epoch 1/30
 - 154s - loss: 0.4697 - acc: 0.7841
Epoch 2/30
 - 109s - loss: 0.4222 - acc: 0.8171
Epoch 3/30
 - 92s - loss: 0.4024 - acc: 0.8321
Epoch 4/30
 - 152s - loss: 0.3880 - acc: 0.8359
Epoch 5/30
 - 153s - loss: 0.3772 - acc: 0.8414
Epoch 6/30
 - 156s - loss: 0.3692 - acc: 0.8456
Epoch 7/30
 - 156s - loss: 0.3638 - acc: 0.8495
Epoch 8/30


SystemError: <built-in function Trainer_train_minibatch> returned a result with an error set